**DeepFace** can handle face detection internally, so you don't necessarily need to use an external face detection model like OpenCV's Haar cascade. When you call `DeepFace.analyze()`, it will automatically detect faces if none are provided. This is very convenient since it simplifies the process by eliminating the need for manual face detection. **DeepFace** uses a variety of face detection models (like `opencv`, `mtcnn`, `dlib`, or `retinaface`) internally depending on the configuration or availability. By default, DeepFace automatically chooses the appropriate model based on the environment and the requirements.


In [ ]:
from deepface import DeepFace
import cv2

# Initialize the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # Analyze the image with DeepFace (detect faces and emotions internally)
        #
        # Set enforce_detection=False: 
        #   This argument allows DeepFace to process the image even if no faces are detected.
        #   If no faces are found, the code won't raise an error.
        analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        
        # Loop through all detected faces (if multiple faces are detected)
        for face_analysis in analysis:
            # Get the dominant emotion
            dominant_emotion = face_analysis['dominant_emotion']

            # Get the region of the detected face (as a dictionary, not a tuple)
            face_region = face_analysis['region']
            x, y, w, h = face_region['x'], face_region['y'], face_region['w'], face_region['h']

            # Draw bounding box around the face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Add the emotion label
            cv2.putText(frame, dominant_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    except Exception as e:
        print(f"Error analyzing face: {e}")

    # Display the video feed with detected emotions
    cv2.imshow("Emotion Detection", frame)

    # Exit on 'q' key or if the window is closed
    key = cv2.waitKey(1)
    if key == ord('q') or cv2.getWindowProperty("Emotion Detection", cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
cv2.destroyAllWindows()


Final version of the code. For additional details, refer to `Prototype/FER/Models/Info.md`.

In [3]:
from deepface import DeepFace
import cv2
import os
import time
from collections import deque
import numpy as np

# Define a buffer for recent emotion scores
maxlen = 15
recent_scores = deque(maxlen=maxlen)

def get_log_filename(base_name="logs/emotion_log"):
    """Generate a unique log filename by appending a counter if files already exist."""
    os.makedirs("logs", exist_ok=True)  # Ensure the logs folder exists
    counter = 0
    while True:
        filename = f"{base_name}_{counter}.txt"
        if not os.path.exists(filename):
            return filename
        counter += 1

def process_video(video_file, show_preview=True):
    # Determine video source
    video_source = 0 if video_file == "camera" else video_file

    # Open video capture
    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
        print(f"Error: Could not open video source '{video_source}'.")
        return

    # Generate a unique log filename
    log_filename = get_log_filename()
    log_file = open(log_filename, "w")
    log_file.write("Timestamp,Emotion,Confidence\n")

    # Variables for FPS calculation
    frame_count = 0
    start_time = time.time()

    while True:
        success, frame = cap.read()
        if not success:
            break

        try:
            # Analyze the image with DeepFace (detect faces and emotions internally)
            analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)

            for face_analysis in analysis:
                # Get the dominant emotion and its confidence score
                dominant_emotion = face_analysis['dominant_emotion']
                emotion_scores = face_analysis['emotion']

                # Smooth the scores using a deque
                recent_scores.append(list(emotion_scores.values()))
                smoothed_scores = np.mean(recent_scores, axis=0)
                smoothed_emotion_idx = np.argmax(smoothed_scores)
                smoothed_emotion = list(emotion_scores.keys())[smoothed_emotion_idx]

                # Get the region of the detected face
                face_region = face_analysis['region']
                x, y, w, h = face_region['x'], face_region['y'], face_region['w'], face_region['h']

                if video_file == "camera":
                    # Get the current timestamp
                    timestamp = time.time()
                else:
                    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0  # Convert to seconds

                # Log the emotion and confidence
                log_file.write(f"{timestamp:.2f},{smoothed_emotion},{smoothed_scores[smoothed_emotion_idx]:.2f}\n")

                # Draw bounding box and emotion label (if preview is enabled)
                if show_preview:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, smoothed_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        except Exception as e:
            print(f"Error analyzing frame: {e}")

        # Increment frame count
        frame_count += 1

        # Show the video feed (if preview is enabled)
        if show_preview:
            cv2.imshow("DeepFace Expression Recognition", frame)

            # Break on 'q' key press or window close
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if cv2.getWindowProperty("DeepFace Expression Recognition", cv2.WND_PROP_VISIBLE) < 1:
                break

    # Calculate FPS
    end_time = time.time()
    total_time = end_time - start_time
    fps = frame_count / total_time
    print(f"Processed {frame_count} frames in {total_time:.2f} seconds. FPS: {fps:.2f}")

    # Log the FPS
    log_file.write(f"\nProcessed {frame_count} frames in {total_time:.2f} seconds.\n")
    log_file.write(f"FPS: {fps:.2f}\n")
    log_file.close()

    # Release resources
    cap.release()
    if show_preview:
        cv2.destroyAllWindows()
        cv2.waitKey(1)

if __name__ == '__main__':
    # Replace 'input_video.mp4' with the path to your video file or use "camera" for live feed
    process_video('../ExampleVideos/surry.mp4', show_preview=True)


Processed 523 frames in 62.49 seconds. FPS: 8.37
